In [1]:
%pylab ipympl
import pandas as pd
import scipy.stats as stats
from scipy import interpolate

Populating the interactive namespace from numpy and matplotlib


# Method 1:  Gauss Meter Readings

In [2]:
mu = 4*pi*10**(-7) #Netwons per ampere square 

In [3]:
df1 =pd.read_csv('/Users/jennifer/Desktop/shortmag.csv',sep=',')
df2 = pd.read_csv('/Users/jennifer/Desktop/longmag.csv',sep=',')

In [4]:
zs = array([val/100 for val in df1['Distance ']]) #distance in m
xs = 1/(zs**3)

zl = array([val/100 for val in df2['Distance']]) #distance in m
xl = 1/(zl**3)

In [5]:
Bs = array([val for val in df1['Dat']]); Bs[0] = 150
Bs = Bs * 10**(-4) # B was in Gauss - converting to teslas (T = netwon / Amp metre)

Bl = array([val for val in df2['Dat']])
Bl = Bl * 10**(-4)

In [6]:
Bl?

In [7]:
fits = stats.linregress(xs[14:],Bs[14:]) # [14:]
slopes = fits[0]
bs = fits[1]
goodness = fits[2]
errs = fits[4]

i = 11
fitl = stats.linregress(xl[i:],Bl[i:])
slopel = fitl[0]
bl = fitl[1]
goodnesl = fitl[2]
errl = fitl[4]

print(goodnesl)

0.9925368140694594


In [8]:
def pm_method1( slope , std ):
    pm = slope * ( (2*pi)/(mu) )
    err = (std)*( (2*pi)/(mu) )
    return pm , err

In [9]:
pms = pm_method1(slopes,errs)

pml = pm_method1(slopel,errl)

print("Short pm: {} \n Long pm: {:}".format(pms,pml))

Short pm: (0.33795992725803276, 0.01459211800357053) 
 Long pm: (0.3868115884388539, 0.013180943064183458)


In [10]:
close(1);figure(1)
plot(xs[14:],Bs[14:],'-o',label='Experimental data')
xlines = linspace(1500,2500)
plot( xlines , slopes*xlines + bs , '--' ,label='Linear fit')

xlabel('Inverse distance from the magnet cubed (m$^{-3}$)'); ylabel('Magnetic field strength (gauss)')
title('Magnetic field strength as a function of inverse distance cubed')
text(1600,0.00016,'y = {:.3g}x + {:.3g}'.format(slopes,bs))
legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
close(2);figure(2)
plot(xl[i:],Bl[i:],'-o',label='Experimental data')
xlinel = linspace(200,4500)
plot( xlinel , slopel*xlinel + bl , '--' ,label='Linear fit')

xlabel('Inverse distance from the magnet cubed (m$^{-3}$)'); ylabel('Magnetic field strength (gauss)')
title('Magnetic field strength as a function of inverse distance cubed')
text(1600,0.00016,'y = {:.3g}x + {:.3g}'.format(slopel,bl))
legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Method 2:  Coil Drop 

### Parameters from the quadratic fit 

In [37]:
A = 4.880 # 1/2 * g
B = -0.008201 # v_0
C = -0.6478 # z_0
g_exp = 2*A

In [62]:
def accel(A , dA):
    g = 2*A
    dg = 2*dA

    return g , dg

In [64]:
print(accel(4.805,0.1519)) #SHORT
print(accel(4.880,0.1460)) #LONG

(9.61, 0.3038)
(9.76, 0.292)


In [38]:
def position(t):
    
    a  = g_exp
    v_t0 = B
    z_t0 = C

    return (1/2)*a*(t**2) + v_t0*t + z_t0

def velocity(t):
    a  = g_exp
    v_t0 = 3.623
    #z_t0 = -0.00775
    z_t0 = 0

    
    return v_t0 + a*t

In [39]:
def emf( t , pm ):
    
    a  = g_exp
    v_t0 = B
    z_t0 = C
    
    constants = (-3/2)*(20)*(mu)*(pm)*(0.0442**2)
    position = (1/2)*a*(t**2) + v_t0*t + z_t0
    velocity = v_t0 + a*t
    

    numerator = constants*position*velocity
    denominator = ( 0.0442**2 + (position**2) )**(5/2)
    
    
    emf = numerator/denominator

    
    return emf 

In [16]:
t_runs = linspace(-0.5,1,10000)

In [33]:
df =pd.read_csv('/Users/jennifer/Desktop/33.csv',sep=',')
times = array([ val for val in df['Latest: Time (s)']])
potBs = array([ val for val in df['Latest: pot 2 / gain']])

In [42]:
dfl = pd.read_csv('/Users/jennifer/Desktop/long.csv',sep=',')
timel = array([ val for val in dfl['Latest: Time (s)']])
potBl = array([ val for val in dfl['Latest: Pot 2/gain']])

In [34]:
from scipy import optimize

In [35]:
opts = optimize.curve_fit(emf,times,potential,method='lm')
opt_pms = opts[0][0]
opt_err = np.sqrt(np.diag(opts[1]))
print(opt_pms,opt_err)

0.466579254975285 [0.00061398]


In [44]:
optl = optimize.curve_fit(emf,timel,potBl,method='lm')
opt_pml = optl[0][0]
opt_errl = np.sqrt(np.diag(optl[1]))
print(opt_pml,opt_errl)

0.46970204645341074 [0.01111759]


In [52]:
close(3);figure(3)

plot(t_run,emf(t_run , pms[0]),label='Theoretical EMF')
plot(t_run,emf(t_run , opt_pms),color='green',label='Optimized Theoretical EMF')

plot( times, potBs , '--', color='orange', label='Experimental EMF')


grid()
legend()

hlines(0,-0.5,1)
xlim(-0.05,0.05)

xlabel('Time (s)'); ylabel('EMF')
title('Theoretical EMF curves superimposed on the experimental EMF curve')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Theoretical EMF curves superimposed on the experimental EMF curve')

In [59]:
close(6);figure(6)

plot(t_run,emf(t_run , pml[0] ),label='Theoretical EMF')
plot(t_run,emf(t_run , opt_pml),color='green',label='Optimized Theoretical EMF')

plot( timel, potBl , '--', color='orange', label='Experimental EMF - Long Magnet')


grid()
legend()

hlines(0,0.3,0.45)
xlim(0.3,0.45)

xlabel('Time (s)'); ylabel('EMF (V)')
title('Theoretical EMF curves superimposed on the experimental EMF')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Theoretical EMF curves superimposed on the experimental EMF')

# Flux and Integration

In [21]:
df =pd.read_csv('/Users/jennifer/Desktop/33.csv',sep=',')
times = array([ val for val in df['Latest: Time (s)']])
potA = array([ val for val in df['Latest: pot 1 / gain']])
potB = array([ val for val in df['Latest: pot 2 / gain']])
potC = array([ val for val in df['Latest: Pot 3 / gain']])
potD = array([ val for val in df['Latest: Pot 4 / Gain']])

In [22]:
D = 380
posflux_timeD = times[:D]
posflux_potD = potD[:D]

negflux_timeD = times[D-1:]
negflux_potD = potD[D-1:]

In [23]:
C = 596
posflux_timeC = times[:C]
posflux_potC = potC[:C]

negflux_timeC = times[C-1:]
negflux_potC = potC[C-1:]

In [24]:
B = 751
posflux_timeB = times[:B]
posflux_potB = potB[:B]

negflux_timeB = times[B-1:]
negflux_potB = potB[B-1:]

In [25]:
A = 880
posflux_timeA = times[:A]
posflux_potA = potA[:A]

negflux_timeA = times[A-1:]
negflux_potA = potA[A-1:]

In [32]:
close(5);figure(5)
plot(posflux_timeC,posflux_potC)
plot(negflux_timeC,negflux_potC)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
close(4);figure(4)
#plot(times,potential)
#plot(posflux_time,posflux_pot,'o')
plot(times,potA,'o',label='A')
plot(times,potB,'o',label='B')
plot(times,potC,'o',label='C')
plot(times,potD,'o',label='D')
legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
from scipy.integrate import cumtrapz

In [29]:
print(cumtrapz(posflux_potD)[-1])
print(cumtrapz(posflux_potC)[-1])
print(cumtrapz(posflux_potB)[-1])
print(cumtrapz(posflux_potA)[-1])

0.25496961451807376
0.290237529062024
0.2712797746357457
0.2711884943334559


In [30]:
print(cumtrapz(negflux_potD)[-1])
print(cumtrapz(negflux_potC)[-1])
print(cumtrapz(negflux_potB)[-1])
print(cumtrapz(negflux_potA)[-1])

-0.23806216403348776
-0.18258535590778965
-0.26957133950339424
-0.2833659596001137


Positive flux increases  
Negative flux also increases but more   

something weird happened with C  

The difference in pos and neg fluxes for each coil got bigger over time (excpet eith  c)

In [68]:
print( abs(cumtrapz(posflux_potD)[-1] + cumtrapz(negflux_potD)[-1]))
print( abs(cumtrapz(posflux_potC)[-1] + cumtrapz(negflux_potC)[-1]))
print( abs(cumtrapz(posflux_potB)[-1] + cumtrapz(negflux_potB)[-1]))
print( abs(cumtrapz(posflux_potA)[-1] + cumtrapz(negflux_potA)[-1]))

0.016907450484586
0.10765217315423434
0.0017084351323514713
0.012177465266657805
